# Load dataset from NarrativeQA

We use Narrative from Question Generation and RAG Evaluation

In [ ]:
import pandas as pd
from Utils import *
data_loader = DatasetLoader()
from datasets import load_dataset

vector_DB = VectorDatabase()
embedder = Embedder()
data_processor = DataProcessor(embedder=embedder, vectordatabase=vector_DB)

In [ ]:
def extract_narrativeqa_text(split='train'):
    # 載入數據集
    dataset = load_dataset("deepmind/narrativeqa", split=split)
    
    # 用於存儲唯一文本的字典
    unique_summaries = {}
    unique_documents = {}
    
    total_summary_chars = 0
    total_document_chars = 0
    
    # 用於存儲問題和答案的列表
    questions = []
    answers = []
    
    # 從每個示例中提取文本
    for example in dataset:
        summary = example['document']['summary']['text']
        document = example['document']['text']
        metadata = example['document']['kind'] + "\\" + example['document']['summary']['title']
        
        # 只有當摘要和文檔都是唯一的時才添加到 df_doc
        if summary not in unique_summaries and document not in unique_documents:
            unique_summaries[summary] = metadata
            unique_documents[document] = metadata
            total_summary_chars += len(summary)
            total_document_chars += len(document)
        
        # 總是添加問題和答案到 df_qa
        questions.append(example['question']['text'])
        answers_text = ""
        for answer in example['answers']:
            answers_text += answer['text'] + " "
        answers.append(answers_text)
            
    # 從字典創建列表
    summaries = list(unique_summaries.keys())
    documents = list(unique_documents.keys())
    metadata = [unique_summaries[s] for s in summaries]  # 將元數據與摘要對齊
    
    # 計算平均值
    num_examples = len(summaries)
    avg_summary_chars = total_summary_chars / num_examples if num_examples > 0 else 0
    avg_document_chars = total_document_chars / num_examples if num_examples > 0 else 0
    
    # 創建 df_doc DataFrame
    df_doc = pd.DataFrame({
        'summary': summaries,
        'document': documents,
        'metadata': metadata
    })
    
    # 創建 df_qa DataFrame
    df_qa = pd.DataFrame({
        'questions': questions,
        'ground_truths': answers,
        'answers': ['' for _ in range(len(questions))],
        'context': ['' for _ in range(len(questions))]
    })
    
    print(f'唯一文檔數量: {num_examples}')
    print(f'問答對數量: {len(df_qa)}')
    print(f'平均摘要長度: {avg_summary_chars:.2f} 字符')
    print(f'平均文檔長度: {avg_document_chars:.2f} 字符')
    
    return df_doc, df_qa

In [ ]:
df_doc, df_qa = extract_narrativeqa_text(split="train")
df_doc_sample = df_doc.sample(frac=0.01)
df_qa_sample = df_qa.sample(frac=0.01)
df_doc_sample.to_parquet(".parquet/narrative_qa_doc_sample_11.parquet")
df_qa_sample.to_parquet(".parquet/narrative_qa_qa_sample_11.parquet")
df_doc_sample

In [ ]:
# load parquet
df_doc_sample = pd.read_parquet(".parquet/narrative_qa_doc_sample_11.parquet")
df_qa_sample = pd.read_parquet(".parquet/narrative_qa_qa_sample_11.parquet")
df_qa_sample



# Transform the dataframe into .txts

In [ ]:

import re
from bs4 import BeautifulSoup
import unicodedata

def preprocess_content(content: str) -> str:
    soup = BeautifulSoup(content, 'html.parser')
    text = soup.get_text()

    # unify to NFKC normalization form
    text = unicodedata.normalize('NFKC', text)

    # remove url
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)

    # remove extra whitespace
    text = re.sub(r'[ \t]+', ' ', text).strip()

    # remove special characters, but keep some punctuation
    text = re.sub(r'[^\w\s.,!?;:()"-]', '', text)

    # unify quotes
    text = text.replace('"', '"').replace('"', '"')

    # remove consecutive punctuation
    text = re.sub(r'([.,!?;:])\1+', r'\1', text)

    # ensure there is appropriate whitespace between sentences
    text = re.sub(r'([.,!?;:])\s*', r'\1 ', text)

    return text.strip()

def write_text_to_files_by_metadata(df):
    # Ensure the dataframe has the required columns
    if not all(col in df.columns for col in ['document', 'metadata']):
        raise ValueError("Dataframe must contain 'document' and 'metadata' columns")

    # Dictionary to keep track of file handles
    file_handles = {}

    try:
        for _, row in df.iterrows():
            metadata = row['metadata']
            document = preprocess_content(row['document'])

            # Create or get file handle
            if metadata not in file_handles:
                filename = f".txt/{metadata}.txt"
                file_handles[metadata] = open(filename, 'a', encoding='utf-8')

            # Write document to file
            file_handles[metadata].write(document + "\n\n")  # Add two newlines for separation

    finally:
        # Close all file handles
        for handle in file_handles.values():
            handle.close()

    print(f"Files created: {', '.join(f'{metadata}.txt' for metadata in file_handles.keys())}")

In [ ]:
write_text_to_files_by_metadata(df_doc_sample)

# Embedder into Milvus (GPU) for txts

In [ ]:
data_processor.directory_files_process("narrative_qa_sample_gpu", ".txt/", True, True)

# Import GRAPH RAG data to neo4j

In [ ]:
knowledge_DB = KnowledgeGraphDatabase()

In [ ]:
knowledge_DB.transform_graph_rag_to_neo4j(datapath="../graph_rag_sample/output/20240906-153334/artifacts")

In [ ]:
retriever = Retriever()
retriever.global_retrieve(0)

# Test Modular RAG

In [ ]:
import pandas as pd
rag_evaluation_dataset = pd.read_parquet(".parquet/narrative_qa_qa_sample_11.parquet")
queries = rag_evaluation_dataset["questions"].tolist()[:2]
print(queries)
# print(vector_DB.list_collections())




In [1]:
from Module import *
from Config.output_pydantic import *
workflow = WorkFlowModularHybridRAG()

results = workflow.graph.stream({
    "repeat_times": 0,
    "specific_collection": "narrative_qa_standard_gpu",
    "user_query": "What is the main topic of this dataset?",
    "all_results": [],
    "all_contexts": [[]],
})

for result in results:
    print(result)

/opt/anaconda3/envs/poetry310/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)
2024-09-11 14:32:08,068 - 137473468536320 - milvus_client.py-milvus_client:658 - DEBUG: Created new connection using: 300cfc5147644c23a064d29039cc2c6a


Initializing sparse embedder...
Embedder initialized
Connected to Milvus at localhost:19530 with database default.
VectorDatabase initialized.
GraphDatabase initialized.
Retriever initialized
Agents initialized (with model: "gpt-4o-mini" and temperature: "0.1")
Tasks initialized
MultiAgent RAG System initialized
 [2024-09-11 14:32:08][DEBUG]: == Working Agent: Query Classifier
 [2024-09-11 14:32:08][INFO]: == Starting Task: 
Analyze the following user query and determine if it requires information retrieval to be answered accurately, while also evaluating its global or local scope:

User Query: "What is the main topic of this dataset?"

Your task is to:
1. Classify this query as either requiring retrieval or not
2. Evaluate the query's domain range score

Consider the following guidelines:

1. Queries that typically require retrieval:
   - Specific factual information (e.g., historical dates, statistics, current events)
   - Detailed explanations of complex topics
   - Information abou

2024-09-11 14:32:11,110 - 137464777803456 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed
2024-09-11 14:32:11,114 - 137464735860416 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed
2024-09-11 14:32:11,116 - 137464735860416 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed
2024-09-11 14:32:11,119 - 137464735860416 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed
2024-09-11 14:32:11,121 - 137464735860416 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-09-11 14:32:11][DEBUG]: == [Query Classifier] Task output: needs_retrieval=True domain_range_score=71 justification="The query 'What is the main topic of this dataset?' requires information retrieval because it pertains to a specific dataset, which likely contains unique content that cannot be answered with general knowledge. The scope is broad, as understanding the main topic of a dataset typically involves analyzing its contents, which may vary significantly across different datasets. Therefore, it falls into the category of requiring detailed factual information and integration of knowledge from various sources."


{'user_query_classification_node': {'user_query_classification_result': UserQueryClassificationResult(needs_retrieval=True, domain_range_score=71, justification="The query 'What is the main topic of this dataset?' requires information retrieval because it pertains to a specific dataset, which likely contains unique content that cannot be answered with general knowl

2024-09-11 14:32:15,534 - 137464777803456 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


...

 [2024-09-11 14:33:15][DEBUG]: == Working Agent: Reranker
 [2024-09-11 14:33:15][INFO]: == Starting Task: 
Your task is to evaluate each retrieved data item's relevance to the user's query or sub-queries relevant to the user's query.
-----User Query-----
"What is the main topic of this dataset?"

-----Sub-queries-----
"[]"

Your specific responsibilities are:
1. Compare each data item to the user's query and sub-queries.
2. Assign a relevance score to each data item based on how well it matches the user's query from 0 to 100.
   - Higher scores indicate better relevance.
3. Create a list of these relevance scores, don't include any other information.
4. CRITICAL: Ensure the number of scores in your output list EXACTLY matches the number of data items :5 in the input.

You will receive a list of 5 data items. 
----------------batch_retrieved_data----------------
[{'content': "<b>\t\t\t\tGEORGE\n</b>\t\tYes, he's sitting right there at the end\n\t\tof the table.\n\n<b>\t\t\t\tPROSECUTOR\

2024-09-11 14:33:20,995 - 137464777803456 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-09-11 14:33:20][DEBUG]: == [Reranker] Task output: relevance_scores=[10, 20, 30, 40, 50]


{'detailed_search_node': {'detailed_search_result': DetailedSearchResult(sorted_retrieved_data=[{'content': "<b>\t\t\t\tDIEGO\n</b>\t\tIn my country, I am a magico.  A man\n\t\twith a dream.  A man on the rise.  To\n\t\ttake nothing and make it something,\n\t\tokay?  I have failed my dream, but I\n\t\twill accomplish.  That is why I am in\n\t\tyour country.  Yes, I lose my freedom.\n\t\tBut they do not take my dream.  Do you\n\t\thave a dream, George?\n\n<b>\t\t\t\tGEORGE\n</b>\t\tI would if I could get some sleep.\n\n<b>\t\t\t\tDIEGO\n</b>\t\tYes, you have a dream.  And maybe you\n\t\taccomplish your dream.  But yet you\n\t\tfailed.  Why? <b>\t\t\t\tGEORGE\n</b>\t\tBecause I got caught.\n\n<b>\t\t\t\tDIEGO\n</b>\t\tNo, my brother.\n\n<b>\t\t\t\tGEORGE\n</b>\t\tBecause they caught me?\n\n<b>\t\t\t\tDIEGO\n</b>\t\tYou failed because you had the wrong\n\t\tdream.\n\n\tDiego climbs off his bu

2024-09-11 14:33:52,098 - 137464777803456 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


I now can give a great answer
Final Answer: 

**1. Themes and Topics:**

**A. Drug Trade and Criminality**
- **George's Involvement in Drug Smuggling:**
  - "I was arrested in Chicago with six hundred and sixty pounds of grass. I think that qualifies me."
  - "The first year we made about 100 million dollars between us. It was an expensive operation."
  - "He was very anti-government. He talked about revolution, forming his own country or island, he was looking for power as well as money."
  - "You failed because you had the wrong dream." - Diego to George, indicating the moral complexities of their choices.

**B. Familial Relationships and Redemption**
- **George's Relationship with Family:**
  - "I want my kid out of protective custody. Now. No fucking around."
  - "His mother's disappointment in his criminal lifestyle creates a rift, while his longing to reconnect with Kristina illustrates his desire for redemption."
- **Emotional Conflict:**
  - "George's relationships with his fam

# Test out Retriever for local retriever and global retriever

In [1]:
from Utils import *
retriever = Retriever()
knowledge_DB = KnowledgeGraphDatabase()
# knowledge_DB.create_entity_vector_index()
# knowledge_DB.create_community_weight()
global_result = retriever.global_retrieve(0)
local_result = retriever.local_retrieve(["What is the meaning of life"])

Initializing sparse embedder...
Embedder initialized
Connected to Milvus at localhost:19530 with database default.
VectorDatabase initialized.
GraphDatabase initialized.
Retriever initialized
GraphDatabase initialized.


In [2]:
local_result

{'text_mapping': [' with dirt. Craig\n looks up at her. CRAIG\n You were him last night, werent you? LOTTE\n (quietly)\n Yes. CRAIG\n And he was with her. LOTTE\n We love her, Craig. Im sorry. CRAIG\n We? LOTTE\n Me and John. CRAIG\n Dont forget me. LOTTE\n Well, you have the Maxine action\n figure to play with. Craig looks down at his coffee. LOTTE (CONTD)\n Im sorry. That was nasty. CRAIG\n Life is confusing, isnt it? LOTTE\n Sometimes were forced to make\n hard decisions. (beat)\n Id like for us to stay together, Craig. You know, platonically, if thats possible. I truly value\n our friendship. CRAIG\n I feel that somehow my parents never\n prepared me to make this particular\n decision. Not that I blame them. How\n could they know? Todays world is so\n complicated. (beat)\n No. I have to go away now. Im sorry, Lotte. Im so sorry. CUT TO: INT. CRAIG AND MAXINES OFFICE - MORNING\n\n Craig enters with red-rimmed eyes. Maxine sits at her\n desk, actually looking kind of radiant. MAXINE\

In [3]:
global_result["communities"]

["# The Wedding-Guest and the Ancient Mariner\n\nThis community centers around the interaction between the Wedding-Guest and the Ancient Mariner, highlighting the emotional and narrative dynamics that unfold during their encounter. The Wedding-Guest serves as a bridge between the audience and the Mariner's haunting tale, reflecting themes of fear, curiosity, and transformation amidst the backdrop of a wedding celebration.\n\n## The Wedding-Guest as a narrative conduit\n\nThe Wedding-Guest is a pivotal character who embodies the audience's engagement with the Ancient Mariner's tale. His initial fear and subsequent curiosity reflect the emotional journey that listeners often experience when confronted with unsettling stories. This dynamic illustrates how storytelling can evoke strong emotions and create a connection between the storyteller and the audience. The Wedding-Guest's reactions serve to amplify the Mariner's narrative, making the audience's experience more immersive and impactfu

In [6]:
import Config.constants as const
import json
from MultiAgent import *
from Utils import *

retriever = Retriever()
multi_agent = MultiAgent_RAG()
# all_communities = retriever.global_retrieve(0)["communities"]

# batches = []
# for i in range(0, len(all_communities), const.NODE_BATCH_SIZE):
#     batch_communities = all_communities[i:i + const.NODE_BATCH_SIZE]
#     batches.append({
#         "user_query": "What is the meaning of life",
#         "sub_queries": [],
#         "batch_communities": batch_communities,
#         "batch_size": len(batch_communities),
#     })


# all_scores = multi_agent.topic_reranking_run_batch_async(node_batch_inputs=batches).relevant_scores
# print(all_scores)
# print(len(all_scores))
# print(len(all_communities))

multi_agent.user_query_classification_run(user_query="Why does the author choose to use first-person point of view in this article?")





2024-09-10 14:44:13,780 - 133405013833216 - milvus_client.py-milvus_client:658 - DEBUG: Created new connection using: eb1140a954114038bfee403fbc57e0dc
2024-09-10 14:44:13,829 - 133405013833216 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


Initializing sparse embedder...
Embedder initialized
Connected to Milvus at localhost:19530 with database default.
VectorDatabase initialized.
GraphDatabase initialized.
Retriever initialized
Agents initialized (with model: "gpt-4o-mini" and temperature: "0.1")
Tasks initialized
LLMMA RAG System initialized


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer  
Final Answer: UserQueryClassificationResult(needs_retrieval=False, domain_range_score=36, justification="The query asks about the author's choice of narrative perspective in an article, which can typically be answered through analysis of the text itself rather than requiring external information retrieval. The scope is moderate as it pertains to literary analysis, which may involve some integration of concepts but does not necessitate specific factual lookup.")

> Finished chain.
 Error parsing JSON: Expecting value: line 1 column 1 (char 0). Attempting to handle partial JSON.


UserQueryClassificationResult(needs_retrieval=False, domain_range_score=36, justification="The query asks about the author's choice of narrative perspective in an article, which can typically be answered through analysis of the text itself rather than requiring external information retrieval. The scope is moderate as it pertains to literary analysis, which may involve some integration of concepts but does not necessitate specific factual lookup.")